In [1]:
# RSI PowerZones Weekly
import gta_indicator
import gta_plot
import gta_prices
import gta_statistics
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'IBPO'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
history_start = '2010-09-14'
p1 = 48 # System 1 look back period.
p2 = 24 # System 2 look back period.
p3 = 12 # System 3 look back period.
p4 = 6  # System 4 look back period.
look_back = (p1, p2, p3, p4)
strategy = f'Donchian'
position_size = 7500  # Position size in major currency unit.
charges = 11.95  # Commission per trade.
stamp_duty = 0.0  # Stamp Duty percentage.
stop_loss = 20  # Stop Loss percentage.

In [3]:
# Function definitions.
def state_signal(entry_signal, exit_signal, period):
    '''Calculate trade state signals.'''
    df = pd.concat([entry_signal, exit_signal], axis=1)
    df.columns = ['entry', 'exit']
    df['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'entry'] == 1 \
        and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'exit'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
    return df.state


def trade_list(close, entry_flag, exit_flag):
    '''Generate trade list'''
    frame = pd.concat([close, entry_flag, exit_flag], axis=1)
    frame.columns = ['close', 'entry_flag', 'exit_flag']
    entry_price = frame.close[frame.entry_flag == 1]
    exit_price = frame.close[frame.exit_flag == 1]
    entry_count = len(entry_price)
    exit_count = len(exit_price)
    if exit_count != entry_count:
        last_price = pd.Series(data=frame.close[-1:], index=frame.index[-1:])
        exit_price = exit_price.append(last_price)
        exit_count = len(exit_price)
    df = pd.DataFrame(columns=['entry_price', 'exit_date', 'exit_price'])
    df.entry_price = entry_price
    df.exit_date = exit_price.index
    df.exit_price = exit_price.values
    return df

In [4]:
# Import weekly closing prices.
df = gta_prices.weekly(exchange, tidm, history_start)

In [5]:
# Calculate Donchian channels.
for period in look_back:
    dc = gta_indicator.donchian(df, period)
    df = df.join(dc)
df

,open,high,low,close,upr48,lwr48,mid48,upr24,lwr24,mid24,upr12,lwr12,mid12,upr6,lwr6,mid6
date,,,,,,,,,,,,,,,,
2010-09-17,1.225,1.400,1.200,1.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-24,1.375,1.375,1.350,1.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-01,1.375,1.400,1.350,1.400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-08,1.395,1.540,1.395,1.540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-15,1.540,1.550,1.540,1.540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,3.910,4.000,3.780,3.860,3.94,2.48,3.21,3.94,2.48,3.21,3.94,3.04,3.49,3.94,3.34,3.64
2022-04-01,3.980,4.000,3.840,3.860,4.00,2.48,3.24,4.00,2.54,3.27,4.00,3.04,3.52,4.00,3.34,3.67
2022-04-08,3.950,3.990,3.820,3.920,4.00,2.48,3.24,4.00,2.54,3.27,4.00,3.04,3.52,4.00,3.34,3.67


In [6]:
# System 1 long trades.

# Check for close above upper Donchian channel.
# df['bto'] = np.where(df.close > df.upr, 1, 0)

# Check for close below mid Donchian channel.
# df['stc'] = np.where(df.close < df.mid, 1, 0)

# Trade state signal.
# df['ss'] = state_signal(df.bto, df.stc, p)
# pd.set_option('display.max_rows', None)

# Trade entry & exit flags.
# df['en1'] = np.where(np.logical_and(df.ss == 1, df.ss.shift(periods=1) == 0), 1, 0)
# df['ex1'] = np.where(np.logical_and(df.ss == 0, df.ss.shift(periods=1) == 1), 1, 0)

# df.loc['2018':'2021']

In [7]:

# Trade list.
# td = trade_list(df.close, df.en1, df.ex1)  # Trade List #1.
# td = td.sort_index()  # Sort trades in ascending date order.
# td